In [2]:
# files and system
import os
import sys

In [3]:
import numpy as np
import matplotlib.pyplot as plt

In [4]:
# working with images
import cv2
import imageio
from PIL import Image
import scipy.ndimage
import skimage.transform

In [5]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F

import torchsummary

# Global Variables

In [6]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu") # select device for training, i.e. gpu or cpu

# EDA

## Mask channels

In [25]:
img = imageio.imread('../datasets/train-val/images/cju5buy2bal250818ipl6fqwv.jpg')
img.shape

(430, 569, 3)

In [8]:
mask = imageio.imread('../datasets/train-val/masks/cju5buy2bal250818ipl6fqwv.jpg')
mask.shape

(430, 569, 3)

In [11]:
print((mask[:, :, 0] == mask[:, :, 1]).all())
print((mask[:, :, 1] == mask[:, :, 2]).all())

True
True


## Pre-processing flow

In [17]:
img = imageio.imread('../datasets/train-val/images/cju5buy2bal250818ipl6fqwv.jpg') / 255
mask = imageio.imread('../datasets/train-val/masks/cju5buy2bal250818ipl6fqwv.jpg')[:, :, 0] / 255
mask = mask.round() # binarize to 0 or 1 

In [18]:
img = torch.FloatTensor(np.transpose(img, [2, 0 ,1]))
mask = torch.FloatTensor(mask).unsqueeze(0)

In [20]:
img.shape, mask.shape

(torch.Size([3, 430, 569]), torch.Size([1, 430, 569]))

In [23]:
sample = torch.cat((img, mask), 0)
sample.shape

4

In [24]:
img.shape[0]

3

## Mask values

In [39]:
np.unique(mask)

Array([  0,   1,   2,   3,   4,   5,   6,   7, 248, 249, 250, 251, 252,
       253, 254, 255], dtype=uint8)

In [40]:
mask = (mask / 255).round()
np.unique(mask)

Array([0., 1.])

## Pre-defined split

In [6]:
os.listdir('../2020-MediaEval-Medico-polyp-segmentation-master/kvasir-seg-train-val')

['val.txt', 'train.txt', 'README.md']

In [7]:
with open('../2020-MediaEval-Medico-polyp-segmentation-master/kvasir-seg-train-val/train.txt', 'r') as f:
    ids_train = [l.strip()+'.jpg' for l in f]
with open('../2020-MediaEval-Medico-polyp-segmentation-master/kvasir-seg-train-val/val.txt', 'r') as f:
    ids_val = [l.strip()+'.jpg' for l in f]

# Normalization constants

In [8]:
path_images = "../datasets/train-val/images"
path_masks = "../datasets/train-val/masks"

In [16]:
means = np.zeros((len(ids_train), 3))
#stds = np.zeros((len(ids_train), 3))

images = np.zeros()

for i, img_id in enumerate(ids_train):
    path_img = os.path.join(path_images, img_id)
    img = imageio.imread(path_img) / 255
    
    means[i, :] = img.mean((0, 1))
    stds[i, :] = img.std((0, 1))

    
print(means.mean(0))
print(stds.mean(0))

[0.55894186 0.32169285 0.23561552]
[0.30564136 0.21442204 0.17755629]


In [20]:
means.mean(0).round(4)

array([0.5589, 0.3217, 0.2356])

In [14]:
stds.mean(0).round(4)

NameError: name 'stds' is not defined

In [ ]:
_means = [0.5589, 0.3217, 0.2356]
_stds= [0.3056, 0.2144, 0.1776]